In [1]:
import pandas as pd
import numpy as np
import re

In [2]:
df = pd.read_parquet("hf://datasets/shahules786/PoetryFoundationData/data/train-00000-of-00001-486832872ed96d17.parquet")
print(f"\n\n======\n\n")

print(df.columns, len(df))





Index(['poem name', 'content', 'author', 'type', 'age'], dtype='object') 13854


In [3]:
def load_embedding_model():
    """ Load GloVe Vectors
        Return:
            wv_from_bin: All 400000 embeddings, each length 50
    """
    import gensim.downloader as api
    wv_from_bin = api.load("glove-wiki-gigaword-50")
    # wv_from_bin = api.load("glove.6B/glove.6B.50d.txt")
    print("Loaded vocab size %i" % len(list(wv_from_bin.index_to_key)))
    return wv_from_bin
wv_from_bin = load_embedding_model()

Loaded vocab size 400000


In [1]:
def process_poem(poem) :
  out = poem.replace('\n', ' ')
  out = out.lower()
  out = re.sub(r'[^a-zA-Z ]+', '', out)
  return [word for word in out.split(' ') if word!='']

processed_poems = [process_poem(df['content'].iloc[i]) for i in range(len(df))]
print(processed_poems[0])
#newyork_labels = [0 for i in range(len(df))]

# processed_poems = newyork_processed + shake_processed
# labels = newyork_labels + shake_labels
# # perm = np.random.permutation(len(processed_poems))
# # processed_poems = processed_poems[perm]
# # labels = labels[perm]

vocab = sorted(list(set([word for poem in processed_poems for word in poem])))

word_to_idx = {word:idx for idx, word in enumerate(vocab)}
idx_to_word = {idx:word for idx, word in enumerate(vocab)}
input_sequences = []
labels = []

for poem in processed_poems :
  for i in range(len(poem)) :
    poem[i] = word_to_idx[poem[i]]
    if i < len(poem) - 1:
      n_gram_sequence = poem[max(0, i-99):i+1]
      input_sequences.append(n_gram_sequence)
    if i > 0:
      labels.append(poem[i])

print(processed_poems[0], idx_to_word[38696])
print(input_sequences[5], labels[5])

# print(processed_poems[0])

embedding_matrix = np.zeros((len(vocab), 50))
bad_count = 0
for i, word in enumerate(vocab):
    try:
        embedding_matrix[i] = wv_from_bin.get_vector(word)
    except:
        print("this is bad", word)
        bad_count += 1
print(f"Total Bad Words: {bad_count} out of total vocab {len(vocab)}")

NameError: name 'df' is not defined

In [5]:
import tensorflow as tf
from tensorflow.keras.preprocessing.sequence import pad_sequences

max_length = max(len(sequence) for sequence in input_sequences)
print(f"Max Length = {max_length}")
predictors = pad_sequences(input_sequences, maxlen=max_length, padding='post', truncating='post')

2024-11-16 08:11:46.116925: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2024-11-16 08:11:46.322110: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory; LD_LIBRARY_PATH: /usr/local/lib:/usr/lib
2024-11-16 08:11:46.322134: I tensorflow/stream_executor/cuda/cudart_stub.cc:29] Ignore above cudart dlerror if you do not have a GPU set up on your machine.
2024-11-16 08:11:46.359165: E tensorflow/stream_executor/cuda/cuda_blas.cc:2981] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2024-11-16 08:11:47.883686

Max Length = 1000


In [2]:
import pickle
import tensorflow as tf

# f = open('predictors.pickle', 'wb')
# pickle.dump(predictors, f)
# f.close()
# f = open('labels.pickle', 'wb')
# pickle.dump(labels, f)
# f.close()

f = open('predictors.pickle', 'rb')
predictors = pickle.load(f)
f.close()
f = open('labels.pickle', 'rb')
labels = pickle.load(f)
f.close()

2024-11-16 07:46:20.326448: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2024-11-16 07:46:21.691856: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory; LD_LIBRARY_PATH: /usr/local/lib:/usr/lib
2024-11-16 07:46:21.691887: I tensorflow/stream_executor/cuda/cudart_stub.cc:29] Ignore above cudart dlerror if you do not have a GPU set up on your machine.
2024-11-16 07:46:21.856517: E tensorflow/stream_executor/cuda/cuda_blas.cc:2981] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2024-11-16 07:46:24.402799

In [ ]:
perm = np.random.permutation(len(predictors))
print("yay")
shuffled_predictors = np.array(predictors)[perm]
shuffled_labels = np.array(labels)[perm]
print('nice')
training_data = shuffled_predictors[:-500000]
training_labels = shuffled_labels[:-500000]
print('cool')
validation_data = shuffled_predictors[-500000:]
validation_labels = shuffled_labels[-500000:]

yay


## LSTM Model

In [7]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, LSTM, Embedding, Dropout

# len(vocab) = 161619
model_lstm = tf.keras.Sequential()
e = Embedding(len(vocab), 50, weights=[embedding_matrix], input_length = 100, trainable=False)  # used to be a trainable embedding layer to 10d
model_lstm.add(e)
model_lstm.add(LSTM(100))
model_lstm.add(Dropout(0.1))
model_lstm.add(Dense(50, activation='sigmoid'))   # used to have len(vocab) neurons
# lr_schedule = tf.keras.optimizers.schedules.InverseTimeDecay(0.1, decay_rate=0.1, decay_steps=1000)
# optimizer = tf.keras.optimizers.Adam(learning_rate=lr_schedule)
model_lstm.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])
print(model_lstm.summary())
print(len(vocab))

2024-11-16 08:15:26.259657: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.


Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding (Embedding)       (None, 1000, 50)          8080950   
                                                                 
 lstm (LSTM)                 (None, 100)               60400     
                                                                 
 dropout (Dropout)           (None, 100)               0         
                                                                 
 dense (Dense)               (None, 50)                5050      
                                                                 
Total params: 8,146,400
Trainable params: 65,450
Non-trainable params: 8,080,950
_________________________________________________________________
None
161619


In [9]:
model_lstm.fit(np.array(training_data), np.array([embedding_matrix[word] for word in training_labels]), epochs=1, verbose=1)

2024-11-16 08:18:09.213761: W tensorflow/core/framework/cpu_allocator_impl.cc:82] Allocation of 11593384000 exceeds 10% of free system memory.


  158/90574 [..............................] - ETA: 15:28:59 - loss: -96.3469 - accuracy: 0.7854

KeyboardInterrupt: 

In [ ]:
loss, accuracy = model_lstm.evaluate(np.array(validation_data), np.array(validation_labels), verbose=1)

In [18]:
print(loss, accuracy)

0.0004358454898465425 1.0
